In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.backends.cudnn as cudnn
from sklearn.metrics import (precision_score, recall_score, f1_score,
                             accuracy_score, balanced_accuracy_score, roc_auc_score)

import os

In [ ]:
dir = "/content/drive/MyDrive/aligned"
matrix_type = "FCgsr"

# Determine device for PyTorch (CUDA GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class E2EBlock(nn.Module):
    def __init__(self, in_planes, planes, example, bias=False):
        super().__init__()
        # Use the 4th dimension of the example input to determine the kernel width/height.
        self.d = example.size(3)
        self.cnn1 = nn.Conv2d(in_planes, planes, (1, self.d), bias=bias)
        self.cnn2 = nn.Conv2d(in_planes, planes, (self.d, 1), bias=bias)

    def forward(self, x):
        a = self.cnn1(x)
        b = self.cnn2(x)
        # Explicit dimension specification for concatenation
        return torch.cat([a]*self.d, dim=3) + torch.cat([b]*self.d, dim=2)

class BrainNetCNN(nn.Module):
    def __init__(self, example):
        super().__init__()
        # Use the example input to determine the number of input channels and spatial dimensions.
        self.in_planes = example.size(1)
        self.d = example.size(3)

        self.E2Econv1 = E2EBlock(1, 32, example, bias=True)
        self.E2Econv2 = E2EBlock(32, 64, example, bias=True)
        self.E2N = nn.Conv2d(64, 1, (1, self.d))
        self.N2G = nn.Conv2d(1, 256, (self.d, 1))
        self.dense1 = nn.Linear(256, 128)
        self.dense2 = nn.Linear(128, 30)
        self.dense3 = nn.Linear(30, 1) # Output a single logit

    def forward(self, x):
        out = F.leaky_relu(self.E2Econv1(x), negative_slope=0.33)
        out = F.leaky_relu(self.E2Econv2(out), negative_slope=0.33)
        out = F.leaky_relu(self.E2N(out), negative_slope=0.33)
        out = F.dropout(F.leaky_relu(self.N2G(out), negative_slope=0.33), p=0.5)
        out = out.view(out.size(0), -1)
        out = F.dropout(F.leaky_relu(self.dense1(out), negative_slope=0.33), p=0.5)
        out = F.dropout(F.leaky_relu(self.dense2(out), negative_slope=0.33), p=0.5)
        out = self.dense3(out)
        return out

# Dataset and DataLoader updates
class NCANDA_Dataset(Dataset):
    def __init__(self, directory=dir, matrix_type=matrix_type, mode="train", transform=None, class_balancing=False):
        """
        Args:
            directory (string): Path to the dataset.
            mode (str): "train" for training, "validation" for validation, "train+validation" for full training.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.directory = directory
        self.mode = mode
        self.transform = transform

        x = np.load(os.path.join(directory, "X_" + matrix_type + "_control_moderate.npy"))
        y = np.load(os.path.join(directory, "y_aligned_control_moderate.npy"))
        print(f"Loaded {matrix_type} data with shape: {x.shape}, {y.shape}")
        print("Amount of 1s in y: ", np.sum(y == 1))

        X_train, X_test, y_train, y_test = train_test_split(
            x, y, test_size=0.33, random_state=42
        ) # test is the validation set (for conditionals below)

        if mode == "train":
            x, y = X_train, y_train
            print("Amount of 1s in y (train): ", np.sum(y == 1))
        elif mode == "validation":
            x, y = X_test, y_test
            print("Amount of 1s in y (validation): ", np.sum(y == 1))
        elif mode == "train+validation":
            pass  # Use full dataset
        else:
            raise ValueError("Invalid mode specified")

        # NORMALIZE DATA
        if mode == "train":
            self.mean, self.std = x.mean(), x.std()
        x = (x - X_train.mean()) / X_train.std()  # Normalize using training statistics

        self.X = torch.FloatTensor(np.expand_dims(x, 1).astype(np.float32))
        self.Y = torch.FloatTensor(y).unsqueeze(1)

        print(f"{self.mode} dataset shape: {self.X.shape}, {self.Y.shape}")

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.Y[idx]
        if self.transform:
            x = self.transform(x)
        return x, y  # Return tuple instead of list

# Training and evaluation updates
def train(epoch, net, trainloader, criterion, optimizer):
    net.train()
    running_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(trainloader)

def test(net, testloader, criterion):
    net.eval()
    test_loss = 0.0
    all_targets = []
    all_logits = []

    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()

            all_logits.append(outputs.cpu())
            all_targets.append(targets.cpu())

    # Concatenate and compute metrics
    all_logits = torch.cat(all_logits, dim=0)
    all_targets = torch.cat(all_targets, dim=0)

    y_prob = torch.sigmoid(all_logits).numpy()  # Convert logits to probabilities
    y_pred = (y_prob > 0.5).astype(int)  # Convert probabilities to binary labels

    y_true = all_targets.numpy()

    accuracy = accuracy_score(y_true, y_pred) * 100
    balanced_acc = balanced_accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_prob)

    avg_loss = test_loss / len(testloader)
    # print(f"Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    # print(f"Balanced Accuracy: {balanced_acc:.2f}%")
    # print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    # print(f"ROC-AUC: {roc_auc:.4f}")

    return avg_loss, accuracy, balanced_acc, precision, recall, f1, roc_auc

# Main execution flow
if __name__ == "__main__":
    # Initialize datasets and loaders
    trainset = NCANDA_Dataset(mode="train")
    testset = NCANDA_Dataset(mode="validation")
    print("Training on matrix type: ", matrix_type)

    trainloader = DataLoader(trainset, batch_size=20, shuffle=True,
                           num_workers=2, pin_memory=True)
    testloader = DataLoader(testset, batch_size=20, shuffle=False,
                          num_workers=2, pin_memory=True)

    # Model setup
    # Initialize the network using an example input from the training set
    net = BrainNetCNN(trainset.X[0:1])
    net = net.to(device)
    if device.type == "cuda":
        net = torch.nn.DataParallel(net)
        cudnn.benchmark = True

    # Initialize weights using Kaiming initialization
    def init_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.kaiming_uniform_(m.weight, nonlinearity='leaky_relu', a=0.33)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
    net.apply(init_weights)

    # Training parameters
    # We have 280 of class 0 and 373 of class 1 (in training data)
    pos_weight = torch.tensor([280 / 373], device=device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = torch.optim.AdamW(net.parameters(), lr=1e-3, weight_decay=1e-2)

    # Training loop
    num_epochs = 200
    train_losses = []
    test_losses = []
    accuracies = []
    balanced_accs = []
    precisions = []
    recalls = []
    f1_scores = []
    roc_aucs = []

    for epoch in range(num_epochs):
        train_loss = train(epoch, net, trainloader, criterion, optimizer)
        test_metrics = test(net, testloader, criterion)
        test_loss, accuracy, balanced_acc, precision, recall, f1, roc_auc = test_metrics

        # Append metrics
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        accuracies.append(accuracy)
        balanced_accs.append(balanced_acc)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)
        roc_aucs.append(roc_auc)

        # Print every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}")
            print(f"Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")
            print(f"Accuracy: {accuracy:.2f}% | Balanced Acc: {balanced_acc:.2f}%")
            print(f"Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f} | ROC-AUC: {roc_auc:.4f}")

    # Save all metrics
    torch.save(net.state_dict(), "brainnetcnn_model.pth")
    np.savez("training_stats.npz",
             train_losses=train_losses,
             test_losses=test_losses,
             accuracies=accuracies,
             balanced_accs=balanced_accs,
             precisions=precisions,
             recalls=recalls,
             f1_scores=f1_scores,
             roc_aucs=roc_aucs)

Loaded FCgsr data with shape: (653, 109, 109), (653,)
Amount of 1s in y:  373
Amount of 1s in y (train):  252
train dataset shape: torch.Size([437, 1, 109, 109]), torch.Size([437, 1])
Loaded FCgsr data with shape: (653, 109, 109), (653,)
Amount of 1s in y:  373
Amount of 1s in y (validation):  121
validation dataset shape: torch.Size([216, 1, 109, 109]), torch.Size([216, 1])
Training on matrix type:  FCgsr
Epoch 10/200
Train Loss: 0.6375 | Test Loss: 0.5914
Accuracy: 52.31% | Balanced Acc: 52.46%
Precision: 0.5849 | Recall: 0.5124 | F1: 0.5463 | ROC-AUC: 0.5535
Epoch 20/200
Train Loss: 0.5724 | Test Loss: 0.6024
Accuracy: 51.85% | Balanced Acc: 52.84%
Precision: 0.5934 | Recall: 0.4463 | F1: 0.5094 | ROC-AUC: 0.5297
Epoch 30/200
Train Loss: 0.5322 | Test Loss: 1.8196
Accuracy: 48.61% | Balanced Acc: 52.55%
Precision: 0.6316 | Recall: 0.1983 | F1: 0.3019 | ROC-AUC: 0.4829
Epoch 40/200
Train Loss: 0.0768 | Test Loss: 4.3853
Accuracy: 50.93% | Balanced Acc: 52.92%
Precision: 0.6027 | Reca

In [ ]:
dir = "/content/drive/MyDrive/aligned"
matrix_type = "SC"


# Determine device for PyTorch (CUDA GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class E2EBlock(nn.Module):
    def __init__(self, in_planes, planes, example, bias=False):
        super().__init__()
        # Use the 4th dimension of the example input to determine the kernel width/height.
        self.d = example.size(3)
        self.cnn1 = nn.Conv2d(in_planes, planes, (1, self.d), bias=bias)
        self.cnn2 = nn.Conv2d(in_planes, planes, (self.d, 1), bias=bias)

    def forward(self, x):
        a = self.cnn1(x)
        b = self.cnn2(x)
        # Explicit dimension specification for concatenation
        return torch.cat([a]*self.d, dim=3) + torch.cat([b]*self.d, dim=2)

class BrainNetCNN(nn.Module):
    def __init__(self, example):
        super().__init__()
        # Use the example input to determine the number of input channels and spatial dimensions.
        self.in_planes = example.size(1)
        self.d = example.size(3)

        self.E2Econv1 = E2EBlock(1, 32, example, bias=True)
        self.E2Econv2 = E2EBlock(32, 64, example, bias=True)
        self.E2N = nn.Conv2d(64, 1, (1, self.d))
        self.N2G = nn.Conv2d(1, 256, (self.d, 1))
        self.dense1 = nn.Linear(256, 128)
        self.dense2 = nn.Linear(128, 30)
        self.dense3 = nn.Linear(30, 1) # Output a single logit

    def forward(self, x):
        out = F.leaky_relu(self.E2Econv1(x), negative_slope=0.33)
        out = F.leaky_relu(self.E2Econv2(out), negative_slope=0.33)
        out = F.leaky_relu(self.E2N(out), negative_slope=0.33)
        out = F.dropout(F.leaky_relu(self.N2G(out), negative_slope=0.33), p=0.5)
        out = out.view(out.size(0), -1)
        out = F.dropout(F.leaky_relu(self.dense1(out), negative_slope=0.33), p=0.5)
        out = F.dropout(F.leaky_relu(self.dense2(out), negative_slope=0.33), p=0.5)
        out = self.dense3(out)
        return out

# Dataset and DataLoader updates
class NCANDA_Dataset(Dataset):
    def __init__(self, directory=dir, matrix_type=matrix_type, mode="train", transform=None, class_balancing=False):
        """
        Args:
            directory (string): Path to the dataset.
            mode (str): "train" for training, "validation" for validation, "train+validation" for full training.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.directory = directory
        self.mode = mode
        self.transform = transform

        x = np.load(os.path.join(directory, "X_" + matrix_type + "_control_moderate.npy"))
        y = np.load(os.path.join(directory, "y_aligned_control_moderate.npy"))
        print(f"Loaded {matrix_type} data with shape: {x.shape}, {y.shape}")
        print("Amount of 1s in y: ", np.sum(y == 1))

        X_train, X_test, y_train, y_test = train_test_split(
            x, y, test_size=0.33, random_state=42
        ) # test is the validation set (for conditionals below)

        if mode == "train":
            x, y = X_train, y_train
            print("Amount of 1s in y (train): ", np.sum(y == 1))
        elif mode == "validation":
            x, y = X_test, y_test
            print("Amount of 1s in y (validation): ", np.sum(y == 1))
        elif mode == "train+validation":
            pass  # Use full dataset
        else:
            raise ValueError("Invalid mode specified")

        # NORMALIZE DATA
        if mode == "train":
            self.mean, self.std = x.mean(), x.std()
        x = (x - X_train.mean()) / X_train.std()  # Normalize using training statistics

        self.X = torch.FloatTensor(np.expand_dims(x, 1).astype(np.float32))
        self.Y = torch.FloatTensor(y).unsqueeze(1)

        print(f"{self.mode} dataset shape: {self.X.shape}, {self.Y.shape}")

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.Y[idx]
        if self.transform:
            x = self.transform(x)
        return x, y  # Return tuple instead of list

# Training and evaluation updates
def train(epoch, net, trainloader, criterion, optimizer):
    net.train()
    running_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(trainloader)

def test(net, testloader, criterion):
    net.eval()
    test_loss = 0.0
    all_targets = []
    all_logits = []

    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()

            all_logits.append(outputs.cpu())
            all_targets.append(targets.cpu())

    # Concatenate and compute metrics
    all_logits = torch.cat(all_logits, dim=0)
    all_targets = torch.cat(all_targets, dim=0)

    y_prob = torch.sigmoid(all_logits).numpy()  # Convert logits to probabilities
    y_pred = (y_prob > 0.5).astype(int)  # Convert probabilities to binary labels

    y_true = all_targets.numpy()

    accuracy = accuracy_score(y_true, y_pred) * 100
    balanced_acc = balanced_accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_prob)

    avg_loss = test_loss / len(testloader)
    # print(f"Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    # print(f"Balanced Accuracy: {balanced_acc:.2f}%")
    # print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    # print(f"ROC-AUC: {roc_auc:.4f}")

    return avg_loss, accuracy, balanced_acc, precision, recall, f1, roc_auc

# Main execution flow
if __name__ == "__main__":
    # Initialize datasets and loaders
    trainset = NCANDA_Dataset(mode="train")
    testset = NCANDA_Dataset(mode="validation")
    print("Training on matrix type: ", matrix_type)

    trainloader = DataLoader(trainset, batch_size=20, shuffle=True,
                           num_workers=2, pin_memory=True)
    testloader = DataLoader(testset, batch_size=20, shuffle=False,
                          num_workers=2, pin_memory=True)

    # Model setup
    # Initialize the network using an example input from the training set
    net = BrainNetCNN(trainset.X[0:1])
    net = net.to(device)
    if device.type == "cuda":
        net = torch.nn.DataParallel(net)
        cudnn.benchmark = True

    # Initialize weights using Kaiming initialization
    def init_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.kaiming_uniform_(m.weight, nonlinearity='leaky_relu', a=0.33)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
    net.apply(init_weights)

    # Training parameters
    # We have 280 of class 0 and 373 of class 1 (in training data)
    pos_weight = torch.tensor([280 / 373], device=device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = torch.optim.AdamW(net.parameters(), lr=1e-3, weight_decay=1e-2)

    # Training loop
    num_epochs = 200
    train_losses = []
    test_losses = []
    accuracies = []
    balanced_accs = []
    precisions = []
    recalls = []
    f1_scores = []
    roc_aucs = []

    for epoch in range(num_epochs):
        train_loss = train(epoch, net, trainloader, criterion, optimizer)
        test_metrics = test(net, testloader, criterion)
        test_loss, accuracy, balanced_acc, precision, recall, f1, roc_auc = test_metrics

        # Append metrics
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        accuracies.append(accuracy)
        balanced_accs.append(balanced_acc)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)
        roc_aucs.append(roc_auc)

        # Print every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}")
            print(f"Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")
            print(f"Accuracy: {accuracy:.2f}% | Balanced Acc: {balanced_acc:.2f}%")
            print(f"Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f} | ROC-AUC: {roc_auc:.4f}")

    # Save all metrics
    torch.save(net.state_dict(), "SC_brainnetcnn_model.pth")
    np.savez("SC_training_stats.npz",
             train_losses=train_losses,
             test_losses=test_losses,
             accuracies=accuracies,
             balanced_accs=balanced_accs,
             precisions=precisions,
             recalls=recalls,
             f1_scores=f1_scores,
             roc_aucs=roc_aucs)

Loaded SC data with shape: (653, 90, 90), (653,)
Amount of 1s in y:  373
Amount of 1s in y (train):  252
train dataset shape: torch.Size([437, 1, 90, 90]), torch.Size([437, 1])
Loaded SC data with shape: (653, 90, 90), (653,)
Amount of 1s in y:  373
Amount of 1s in y (validation):  121
validation dataset shape: torch.Size([216, 1, 90, 90]), torch.Size([216, 1])
Training on matrix type:  SC
Epoch 10/200
Train Loss: 0.6283 | Test Loss: 0.5974
Accuracy: 47.69% | Balanced Acc: 47.65%
Precision: 0.5370 | Recall: 0.4793 | F1: 0.5066 | ROC-AUC: 0.5115
Epoch 20/200
Train Loss: 0.5982 | Test Loss: 0.6172
Accuracy: 46.76% | Balanced Acc: 47.16%
Precision: 0.5300 | Recall: 0.4380 | F1: 0.4796 | ROC-AUC: 0.4483
Epoch 30/200
Train Loss: 0.6055 | Test Loss: 0.6021
Accuracy: 51.85% | Balanced Acc: 50.69%
Precision: 0.5659 | Recall: 0.6033 | F1: 0.5840 | ROC-AUC: 0.5134
Epoch 40/200
Train Loss: 0.6130 | Test Loss: 0.5973
Accuracy: 52.78% | Balanced Acc: 51.52%
Precision: 0.5725 | Recall: 0.6198 | F1: 

In [ ]:
dir = "/content/drive/MyDrive/aligned"
matrix_type = "FC"


# Determine device for PyTorch (CUDA GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class E2EBlock(nn.Module):
    def __init__(self, in_planes, planes, example, bias=False):
        super().__init__()
        # Use the 4th dimension of the example input to determine the kernel width/height.
        self.d = example.size(3)
        self.cnn1 = nn.Conv2d(in_planes, planes, (1, self.d), bias=bias)
        self.cnn2 = nn.Conv2d(in_planes, planes, (self.d, 1), bias=bias)

    def forward(self, x):
        a = self.cnn1(x)
        b = self.cnn2(x)
        # Explicit dimension specification for concatenation
        return torch.cat([a]*self.d, dim=3) + torch.cat([b]*self.d, dim=2)

class BrainNetCNN(nn.Module):
    def __init__(self, example):
        super().__init__()
        # Use the example input to determine the number of input channels and spatial dimensions.
        self.in_planes = example.size(1)
        self.d = example.size(3)

        self.E2Econv1 = E2EBlock(1, 32, example, bias=True)
        self.E2Econv2 = E2EBlock(32, 64, example, bias=True)
        self.E2N = nn.Conv2d(64, 1, (1, self.d))
        self.N2G = nn.Conv2d(1, 256, (self.d, 1))
        self.dense1 = nn.Linear(256, 128)
        self.dense2 = nn.Linear(128, 30)
        self.dense3 = nn.Linear(30, 1) # Output a single logit

    def forward(self, x):
        out = F.leaky_relu(self.E2Econv1(x), negative_slope=0.33)
        out = F.leaky_relu(self.E2Econv2(out), negative_slope=0.33)
        out = F.leaky_relu(self.E2N(out), negative_slope=0.33)
        out = F.dropout(F.leaky_relu(self.N2G(out), negative_slope=0.33), p=0.5)
        out = out.view(out.size(0), -1)
        out = F.dropout(F.leaky_relu(self.dense1(out), negative_slope=0.33), p=0.5)
        out = F.dropout(F.leaky_relu(self.dense2(out), negative_slope=0.33), p=0.5)
        out = self.dense3(out)
        return out

# Dataset and DataLoader updates
class NCANDA_Dataset(Dataset):
    def __init__(self, directory=dir, matrix_type=matrix_type, mode="train", transform=None, class_balancing=False):
        """
        Args:
            directory (string): Path to the dataset.
            mode (str): "train" for training, "validation" for validation, "train+validation" for full training.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.directory = directory
        self.mode = mode
        self.transform = transform

        x = np.load(os.path.join(directory, "X_" + matrix_type + "_control_moderate.npy"))
        y = np.load(os.path.join(directory, "y_aligned_control_moderate.npy"))
        print(f"Loaded {matrix_type} data with shape: {x.shape}, {y.shape}")
        print("Amount of 1s in y: ", np.sum(y == 1))

        X_train, X_test, y_train, y_test = train_test_split(
            x, y, test_size=0.33, random_state=42
        ) # test is the validation set (for conditionals below)

        if mode == "train":
            x, y = X_train, y_train
            print("Amount of 1s in y (train): ", np.sum(y == 1))
        elif mode == "validation":
            x, y = X_test, y_test
            print("Amount of 1s in y (validation): ", np.sum(y == 1))
        elif mode == "train+validation":
            pass  # Use full dataset
        else:
            raise ValueError("Invalid mode specified")

        # NORMALIZE DATA
        if mode == "train":
            self.mean, self.std = x.mean(), x.std()
        x = (x - X_train.mean()) / X_train.std()  # Normalize using training statistics

        self.X = torch.FloatTensor(np.expand_dims(x, 1).astype(np.float32))
        self.Y = torch.FloatTensor(y).unsqueeze(1)

        print(f"{self.mode} dataset shape: {self.X.shape}, {self.Y.shape}")

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.Y[idx]
        if self.transform:
            x = self.transform(x)
        return x, y  # Return tuple instead of list

# Training and evaluation updates
def train(epoch, net, trainloader, criterion, optimizer):
    net.train()
    running_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(trainloader)

def test(net, testloader, criterion):
    net.eval()
    test_loss = 0.0
    all_targets = []
    all_logits = []

    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()

            all_logits.append(outputs.cpu())
            all_targets.append(targets.cpu())

    # Concatenate and compute metrics
    all_logits = torch.cat(all_logits, dim=0)
    all_targets = torch.cat(all_targets, dim=0)

    y_prob = torch.sigmoid(all_logits).numpy()  # Convert logits to probabilities
    y_pred = (y_prob > 0.5).astype(int)  # Convert probabilities to binary labels

    y_true = all_targets.numpy()

    accuracy = accuracy_score(y_true, y_pred) * 100
    balanced_acc = balanced_accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_prob)

    avg_loss = test_loss / len(testloader)
    # print(f"Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    # print(f"Balanced Accuracy: {balanced_acc:.2f}%")
    # print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    # print(f"ROC-AUC: {roc_auc:.4f}")

    return avg_loss, accuracy, balanced_acc, precision, recall, f1, roc_auc

# Main execution flow
if __name__ == "__main__":
    # Initialize datasets and loaders
    trainset = NCANDA_Dataset(mode="train")
    testset = NCANDA_Dataset(mode="validation")
    print("Training on matrix type: ", matrix_type)

    trainloader = DataLoader(trainset, batch_size=20, shuffle=True,
                           num_workers=2, pin_memory=True)
    testloader = DataLoader(testset, batch_size=20, shuffle=False,
                          num_workers=2, pin_memory=True)

    # Model setup
    # Initialize the network using an example input from the training set
    net = BrainNetCNN(trainset.X[0:1])
    net = net.to(device)
    if device.type == "cuda":
        net = torch.nn.DataParallel(net)
        cudnn.benchmark = True

    # Initialize weights using Kaiming initialization
    def init_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.kaiming_uniform_(m.weight, nonlinearity='leaky_relu', a=0.33)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
    net.apply(init_weights)

    # Training parameters
    # We have 280 of class 0 and 373 of class 1 (in training data)
    pos_weight = torch.tensor([280 / 373], device=device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = torch.optim.AdamW(net.parameters(), lr=1e-3, weight_decay=1e-2)

    # Training loop
    num_epochs = 200
    train_losses = []
    test_losses = []
    accuracies = []
    balanced_accs = []
    precisions = []
    recalls = []
    f1_scores = []
    roc_aucs = []

    for epoch in range(num_epochs):
        train_loss = train(epoch, net, trainloader, criterion, optimizer)
        test_metrics = test(net, testloader, criterion)
        test_loss, accuracy, balanced_acc, precision, recall, f1, roc_auc = test_metrics

        # Append metrics
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        accuracies.append(accuracy)
        balanced_accs.append(balanced_acc)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)
        roc_aucs.append(roc_auc)

        # Print every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}")
            print(f"Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")
            print(f"Accuracy: {accuracy:.2f}% | Balanced Acc: {balanced_acc:.2f}%")
            print(f"Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f} | ROC-AUC: {roc_auc:.4f}")

    # Save all metrics
    torch.save(net.state_dict(), "FC_brainnetcnn_model.pth")
    np.savez("FC_training_stats.npz",
             train_losses=train_losses,
             test_losses=test_losses,
             accuracies=accuracies,
             balanced_accs=balanced_accs,
             precisions=precisions,
             recalls=recalls,
             f1_scores=f1_scores,
             roc_aucs=roc_aucs)

Loaded FC data with shape: (653, 109, 109), (653,)
Amount of 1s in y:  373
Amount of 1s in y (train):  252
train dataset shape: torch.Size([437, 1, 109, 109]), torch.Size([437, 1])
Loaded FC data with shape: (653, 109, 109), (653,)
Amount of 1s in y:  373
Amount of 1s in y (validation):  121
validation dataset shape: torch.Size([216, 1, 109, 109]), torch.Size([216, 1])
Training on matrix type:  FC
Epoch 10/200
Train Loss: 0.6415 | Test Loss: 0.5956
Accuracy: 49.54% | Balanced Acc: 48.51%
Precision: 0.5476 | Recall: 0.5702 | F1: 0.5587 | ROC-AUC: 0.5126
Epoch 20/200
Train Loss: 0.6106 | Test Loss: 0.6000
Accuracy: 47.22% | Balanced Acc: 49.50%
Precision: 0.5522 | Recall: 0.3058 | F1: 0.3936 | ROC-AUC: 0.5401
Epoch 30/200
Train Loss: 0.6149 | Test Loss: 0.6888
Accuracy: 56.48% | Balanced Acc: 53.13%
Precision: 0.5799 | Recall: 0.8099 | F1: 0.6759 | ROC-AUC: 0.5597
Epoch 40/200
Train Loss: 0.1281 | Test Loss: 2.8973
Accuracy: 48.61% | Balanced Acc: 49.27%
Precision: 0.5521 | Recall: 0.438

In [ ]:
SC_state_dict = torch.load("/content/SC_brainnetcnn_model.pth", map_location=torch.device('cpu'))

print(SC_state_dict.keys())

odict_keys(['module.E2Econv1.cnn1.weight', 'module.E2Econv1.cnn1.bias', 'module.E2Econv1.cnn2.weight', 'module.E2Econv1.cnn2.bias', 'module.E2Econv2.cnn1.weight', 'module.E2Econv2.cnn1.bias', 'module.E2Econv2.cnn2.weight', 'module.E2Econv2.cnn2.bias', 'module.E2N.weight', 'module.E2N.bias', 'module.N2G.weight', 'module.N2G.bias', 'module.dense1.weight', 'module.dense1.bias', 'module.dense2.weight', 'module.dense2.bias', 'module.dense3.weight', 'module.dense3.bias'])


<ipython-input-9-911499977908>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  SC_state_dict = torch.load("/content/SC_brainnetcnn_model.pth", map_location=torch.device('cp